# Creating Customized Flash Cards Using Geo-Location Data

## INTRODUCTION/BUSINESS PROBLEM

    One of my favorite programs is a flashcard app called Anki. It helps me to memorize new information when I study a new language or science topic. It's very efficient at creating strong memories because it has a built-in, research based algorithm which optimizes the time between reviews.
    I've also used Anki to study geography to pretty good effect. It helped me to learn the locations of all the countries in Europe. After this experience I found myself wanting something more customizable: what if I could create a program which would create Anki decks which would help me to learn about what's around me?
    I've moved a lot in life, and every time I get to a new place there is a good amount of time before I can feel my bearings. By getting to know the area better (the venues) I can feel more comfortable. The faster I can know this information, the sooner I can feel at home.
    I want to use foursquare to generate nearby venues given a few paramaters, like location, radius, and venue type, and then create an ordered anki deck from this information. This will help me to get to know my new city better, faster. The audience for this product would be anybody new to a city hoping to learn about what's around them.
    A user would start by specifying their location, desired venues, desired number of cards, and desired attributes. The program would then make a call to Foursquare to find nearby venues, sort them by their type, popularity, or other attribute, and then pass this to a function which would format the dataframe to something able to be imported into Anki. 
    1) User input: location, desired venue type, desired attributes, no. cards to generate
    2) Call to foursquare
    3) Format foursquare data for Anki

## DATA REQUIREMENTS

1) User inputs: location, desired venue type, desired attributes, no. cards to generate 

The user will start by passing in the name of their location to Nominatim, which will generate the location's latitutde/longitude:
```python
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
```

This information can then be used to generate a map of the area using folium:

```python
map = folium.Map(location=[latitude, longitude], zoom_start=11)
```

And then passed to Foursquare's API to get a json file:

```python
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
```

2) Foursquare location data

This data needs to be cleaned and formatted into a pandas dataframe based on the specified attributes given by the user. Once this is done the information can be formatted into something proper to import into Anki